# Feature Engineering
#### Purpose:
The purpose of the file is to perform feature engineering steps on our training and validation parquet files to create a new 'final' training and validation files. These new files will be used in a different notebook for training. No columns will be dropped from the files here, only generation of new features and if needed, scaling.

In [1]:
import os
import pandas as pd
import polars as pl # used for reading in parquet files quickly
import matplotlib as plt
from pathlib import Path
from tqdm import tqdm

### Reading in data

In [2]:
train_df = pl.read_parquet(rf"./Data/train_data.parquet").to_pandas() # convert from polars to pandas 
val_df = pl.read_parquet(rf"./Data/val_data.parquet").to_pandas()

In [3]:
train_df

,student_id,timestamp,question_id,bundle_id,tags,elapsed_time,correct
0,1,1565096190868,5012,3544,74,38000,0.0
1,1,1565096221062,4706,3238,71,24000,1.0
2,1,1565096293432,4366,2898,103,68000,1.0
3,1,1565096339668,4829,3361,83,42000,0.0
4,1,1565096401774,6528,5060,90,59000,0.0
...,...,...,...,...,...,...,...
76078853,840472,1575306027437,9814,7165,136,37000,0.0
76078854,840472,1575306068437,4712,3244,71,37000,0.0
76078855,840472,1575306087437,3793,2325,82,15000,0.0
76078856,840473,1575306037437,3830,2362,106,25000,1.0


In [4]:
val_df

,student_id,timestamp,question_id,bundle_id,tags,elapsed_time,correct
0,4,1566782278107,5177,3709,74,85000,1.0
1,4,1566782311854,8104,5575,8;2;182,28000,1.0
2,4,1566782336708,4291,2823,84,22000,0.0
3,4,1566782351705,4020,2552,86,12000,1.0
4,4,1566782382870,5258,3790,87,28000,1.0
...,...,...,...,...,...,...,...
19215063,840471,1575305806437,10300,7651,76,18000,0.0
19215064,840471,1575305834437,8886,6237,83,25000,0.0
19215065,840471,1575305860437,8556,5907,85,24000,1.0
19215066,840471,1575305880437,9901,7252,74,18000,0.0


## Lecture Tags
#### Purpose:
KT-4 dataset provides information on student interactions. The interaction that we are going to be taking advantage in this case is if a student interacted with a lecture which is denoted by the 'l' char followed by a number. We plan to provide information on whether the student interacted with a lecture associated with a tag for on a question.

In [5]:
def combine_data(file_path, output_file):
    dfs = []
    csv_files = list(Path(file_path).glob("*.csv"))
    
    for csv_file in tqdm(csv_files, desc="Reading CSVs"):
        # getting student_id val
        filename = csv_file.stem # gets name before '.csv' (ex. 'u123')
        student_id = filename[1:] # removes 'u' from name
        print(f"filename: {csv_file}, student_id: {student_id}\n")        
        # scan more memory efficient then read since it does not store
        df = pl.scan_csv(csv_file).with_columns(
            pl.lit(student_id).alias("student_id"))
    
        # add column to df that will represent student_id based of file_num starting at 1
        dfs.append(df)
    
    pl.concat(dfs).sink_parquet(output_file)
    return True # file created successfully

In [6]:
file_path = "./Data/KT3/" 
file = "./Data/combined_kt3.parquet"

if not Path(file).exists():
    result = combine_data(file_path, file)
    if result: print("combined dataset file created successfuly")
    else: print("file failed to create")
else:
    print("combined_kt3.parquet dataset is present in Data folder")

kt3_df = pl.read_parquet("./Data/combined_kt3.parquet").to_pandas()
kt3_df

combined_kt3.parquet dataset is present in Data folder


,timestamp,action_type,item_id,source,user_answer,platform,student_id
0,1565096151269,enter,b3544,diagnosis,None,mobile,1
1,1565096187972,respond,q5012,diagnosis,b,mobile,1
2,1565096194904,submit,b3544,diagnosis,None,mobile,1
3,1565096195001,enter,b3238,diagnosis,None,mobile,1
4,1565096218682,respond,q4706,diagnosis,c,mobile,1
...,...,...,...,...,...,...,...
89270649,1568964975390,enter,b3819,sprint,None,mobile,9998
89270650,1568964992921,respond,q5287,sprint,c,mobile,9998
89270651,1568964996503,submit,b3819,sprint,None,mobile,9998
89270652,1568964996572,enter,e3819,sprint,None,mobile,9998


## Rolling Student Accuracy
We only want to use the student accuracy for questions that they've attempted so far, so this feature tracks the students rolling accuracy up to that point. We use Bayesian smoothing to prevent the students first few interactions from heavily skewing this metric

In [7]:
import polars as pl

train_df = pl.read_parquet(r".\Data\train_data.parquet")
val_df = pl.read_parquet(r".\Data\val_data.parquet")

# Baseline accuracy will be used for Bayesian smoothing - it only uses the training data
baseline_acc = train_df["correct"].mean()
print(f"Baseline accuracy: {baseline_acc:.4f}")

def add_rolling_accuracy(df, baseline_acc):

    alpha = 20 # Strength for Bayesian smoothing

    df = df.sort(["student_id", "timestamp"]).to_pandas()

    # Rolling counts
    df["interaction_number"] = df.groupby("student_id").cumcount()
    df["cumulative_correct"] = df.groupby("student_id")["correct"].cumsum()

    # Past-correct and past-interactions
    df["num_correct"] = df["cumulative_correct"] - df["correct"]
    df["num_interactions"] = df["interaction_number"]

    # Bayesian smoothed accuracy:
    df["bayes_rolling_acc"] = (
        (df["num_correct"] + alpha * baseline_acc) /
        (df["num_interactions"] + alpha)
    )
    df.drop(columns=["cumulative_correct", "num_correct", "interaction_number"], inplace=True)

    return pl.from_pandas(df)

train_df = add_rolling_accuracy(train_df, baseline_acc)
val_df = add_rolling_accuracy(val_df, baseline_acc)

#Prove that the accuracy resets between students
print(train_df.head(1085))

# Temporarily store the new columns to save memory before we read in final dataset
temp_train_num_interactions = train_df["num_interactions"]
temp_train_num_bayes_rolling_acc = train_df["bayes_rolling_acc"]
temp_val_num_interactions = val_df["num_interactions"]
temp_val_num_bayes_rolling_acc = val_df["bayes_rolling_acc"]
del train_df
del val_df

# Read in final datasets to add new features
final_train_df = pl.read_parquet(r".\Data\final_train_data.parquet")
final_val_df = pl.read_parquet(r".\Data\final_val_data.parquet")

final_train_df = final_train_df.with_columns([
    temp_train_num_interactions.alias("num_interactions"),
    temp_train_num_bayes_rolling_acc.alias("bayes_rolling_acc")
])

final_val_df = final_val_df.with_columns([
    temp_val_num_interactions.alias("num_interactions"),
    temp_val_num_bayes_rolling_acc.alias("bayes_rolling_acc")
])

# Save updated final file
final_train_df.write_parquet(r".\Data\final_train_data.parquet")
final_val_df.write_parquet(r".\Data\final_val_data.parquet")

Baseline accuracy: 0.6535
shape: (1_085, 9)
┌────────────┬───────────────┬─────────────┬───────────┬───┬──────────────┬─────────┬─────────────────┬────────────────┐
│ student_id ┆ timestamp     ┆ question_id ┆ bundle_id ┆ … ┆ elapsed_time ┆ correct ┆ num_interaction ┆ bayes_rolling_ │
│ ---        ┆ ---           ┆ ---         ┆ ---       ┆   ┆ ---          ┆ ---     ┆ s               ┆ acc            │
│ i64        ┆ i64           ┆ i64         ┆ i64       ┆   ┆ i64          ┆ f64     ┆ ---             ┆ ---            │
│            ┆               ┆             ┆           ┆   ┆              ┆         ┆ i64             ┆ f64            │
╞════════════╪═══════════════╪═════════════╪═══════════╪═══╪══════════════╪═════════╪═════════════════╪════════════════╡
│ 1          ┆ 1565096190868 ┆ 5012        ┆ 3544      ┆ … ┆ 38000        ┆ 0.0     ┆ 0               ┆ 0.653523       │
│ 1          ┆ 1565096221062 ┆ 4706        ┆ 3238      ┆ … ┆ 24000        ┆ 1.0     ┆ 1               ┆ 0.622

## Lecture Tags
#### Purpose:
KT-4 dataset provides information on student interactions. The interaction that we are going to be taking advantage in this case is if a student interacted with a lecture which is denoted by the 'l' char followed by a number. We plan to provide information on whether the student interacted with a lecture associated with a tag for on a question.

In [8]:
train_df = pl.read_parquet(rf"./Data/train_data.parquet").to_pandas() # convert from polars to pandas 
val_df = pl.read_parquet(rf"./Data/val_data.parquet").to_pandas()

In [9]:
def combine_data(file_path, output_file):
    dfs = []
    csv_files = list(Path(file_path).glob("*.csv"))
    
    for csv_file in tqdm(csv_files, desc="Reading CSVs"):
        # getting student_id val
        filename = csv_file.stem # gets name before '.csv' (ex. 'u123')
        student_id = filename[1:] # removes 'u' from name
        print(f"filename: {csv_file}, student_id: {student_id}\n")        
        # scan more memory efficient then read since it does not store
        df = pl.scan_csv(csv_file).with_columns(
            pl.lit(student_id).alias("student_id"))
    
        # add column to df that will represent student_id based of file_num starting at 1
        dfs.append(df)
    
    pl.concat(dfs).sink_parquet(output_file)
    return True # file created successfully

In [10]:
# read in content data that assoicates lecture_id with tag
contents_df = pd.read_csv("./Data/contents/lectures.csv")
contents_df.columns = contents_df.columns.str.strip()
lec_df = contents_df.drop(['part', 'video_length', 'deployed_at'], axis=1)
lec_df

,lecture_id,tags
0,l520,142
1,l592,142
2,l1259,222
3,l1260,220
4,l1261,221
...,...,...
1016,l839,-1
1017,l840,-1
1018,l841,-1
1019,l842,-1


In [11]:
file_path = "./Data/KT3/" 
file = "./Data/combined_kt3.parquet"

if not Path(file).exists():
    result = combine_data(file_path, file)
    if result: print("combined dataset file created successfuly")
    else: print("file failed to create")
else:
    print("combined_kt3.parquet dataset is present in Data folder")

kt3_df = pl.read_parquet("./Data/combined_kt3.parquet").to_pandas()
kt3_df

combined_kt3.parquet dataset is present in Data folder


,timestamp,action_type,item_id,source,user_answer,platform,student_id
0,1565096151269,enter,b3544,diagnosis,None,mobile,1
1,1565096187972,respond,q5012,diagnosis,b,mobile,1
2,1565096194904,submit,b3544,diagnosis,None,mobile,1
3,1565096195001,enter,b3238,diagnosis,None,mobile,1
4,1565096218682,respond,q4706,diagnosis,c,mobile,1
...,...,...,...,...,...,...,...
89270649,1568964975390,enter,b3819,sprint,None,mobile,9998
89270650,1568964992921,respond,q5287,sprint,c,mobile,9998
89270651,1568964996503,submit,b3819,sprint,None,mobile,9998
89270652,1568964996572,enter,e3819,sprint,None,mobile,9998


In [12]:
cols_drop = ['action_type', 'source', 'user_answer', 'platform']
kt3_df.columns = kt3_df.columns.str.strip()
item_id_df = kt3_df.drop(cols_drop, axis=1)

col_to_filter = 'item_id'
item_filter = 'l' # used to only capture rows in item_id that start w/ 'l' -> represents interacting with lecture
item_lec = item_id_df[item_id_df[col_to_filter].str.contains(item_filter, na=False)]
interact_lec = item_lec.rename(columns={'item_id':'lecture_id'})
interact_lec

,timestamp,lecture_id,student_id
21,1565096637922,l504,1
22,1565097007903,l504,1
1229,1568120237376,l464,1
1230,1568120575616,l464,1
1231,1568120575929,l463,1
...,...,...,...
89255814,1554984988125,l792,9990
89269339,1567424350215,l448,9995
89269340,1567424353598,l448,9995
89269893,1567610491550,l464,9995


In [13]:
combine_df = interact_lec.merge(lec_df[['lecture_id', 'tags']],
                                   on='lecture_id',
                                   how='left')

cols_to_int32 = ['student_id', 'lecture_id']

combine_df['student_id'] = combine_df['student_id'].astype('int32')

print(combine_df['student_id'].dtype)
print(train_df['student_id'].dtype)

combine_df

int32
int64


,timestamp,lecture_id,student_id,tags
0,1565096637922,l504,1,103
1,1565097007903,l504,1,103
2,1568120237376,l464,1,52
3,1568120575616,l464,1,52
4,1568120575929,l463,1,53
...,...,...,...,...
1203605,1554984988125,l792,9990,121
1203606,1567424350215,l448,9995,41
1203607,1567424353598,l448,9995,41
1203608,1567610491550,l464,9995,52


In [14]:
# # Ensure tags are strings for consistent matching
# df_questions['tags'] = df_questions['tags'].astype(str)
# df_lectures['tags'] = df_lectures['tags'].astype(str)

# # Handle multiple tags in questions (split by semicolon)
# df_questions_exploded = df_questions.copy()
# df_questions_exploded['tags'] = df_questions_exploded['tags'].str.split(';')
# df_questions_exploded = df_questions_exploded.explode('tags')
# df_questions_exploded['tags'] = df_questions_exploded['tags'].str.strip()

# # Merge on student_id AND tags
# merged = df_questions_exploded.merge(
#     df_lectures[['student_id', 'tags', 'timestamp']],
#     on=['student_id', 'tags'],
#     how='left',
#     suffixes=('_question', '_lecture')
# )

# # Filter: keep only lectures watched BEFORE the question was answered
# merged['watched_before'] = merged['timestamp_lecture'] < merged['timestamp_question']

# # Group by original question row and check if ANY lecture was watched before
# watched_mask = merged[merged['watched_before'] == True].groupby(level=0).size() > 0

# # Add the column to original dataframe (reindex to handle questions with no match)
# df_questions['watched_lecture'] = watched_mask.reindex(df_questions.index, fill_value=False).astype(int)

# print(df_questions[['student_id', 'question_id', 'tags', 'watched_lecture']].head(10))

In [15]:
# Ensure tags are strings for consistent matching
# train_df['tags'] = train_df['tags'].astype(str)
# combine_df['tags'] = combine_df['tags'].astype(str)

# # Handle multiple tags in questions (split by semicolon)
# train_df_exploded = train_df.copy()
# train_df_exploded['tags'] = train_df_exploded['tags'].str.split(';')
# train_df_exploded = train_df_exploded.explode('tags')
# train_df_exploded['tags'] = train_df_exploded['tags'].str.strip()
# print(train_df_exploded['student_id'].dtype)
# # Merge on student_id AND tags
# merged = train_df_exploded.merge(
#     combine_df[['student_id', 'tags', 'timestamp']],
#     on=['student_id', 'tags'],
#     how='left',
#     suffixes=('_question', '_lecture')
# )

# # Filter: keep only lectures watched BEFORE the question was answered
# merged['watched_before'] = merged['timestamp_lecture'] < merged['timestamp_question']

# # Group by original question row and check if ANY lecture was watched before
# watched_mask = merged[merged['watched_before'] == True].groupby(level=0).size() > 0

# # Add the column to original dataframe (reindex to handle questions with no match)
# final_train_df['watched_lecture'] = watched_mask.reindex(df_questions.index, fill_value=False).astype(int)

# print(train_df[['student_id', 'question_id', 'tags', 'watched_lecture']].head(10))

In [22]:
print(train_df['tags'].dtype)
print(train_df['tags'].head(20))
print(train_df['tags'].isna().sum())

# Find the problematic rows
problematic = train_df[train_df['tags'].isna() | (train_df['tags'] == '')]
print(f"Problematic rows: {len(problematic)}")
print(problematic[['tags']].head())

object
0                       [74]
1                       [71]
2                      [103]
3                       [83]
4                       [90]
5                       [82]
6                       [77]
7     [30, 34, 42, 181, 183]
8         [24, 43, 184, 181]
9     [27, 28, 45, 184, 182]
10    [27, 44, 49, 181, 184]
11        [24, 25, 182, 183]
12        [31, 32, 185, 181]
13    [30, 34, 35, 184, 182]
14    [30, 44, 49, 184, 181]
15    [30, 28, 29, 185, 181]
16        [24, 25, 185, 183]
17                      [72]
18                     [106]
19                     [144]
Name: tags, dtype: object
0
Problematic rows: 0
Empty DataFrame
Columns: [tags]
Index: []


In [ ]:
# Don't explode - use vectorized string operations instead
train_df['tags'] = train_df['tags'].apply(lambda x: x if isinstance(x, list) else [])

watched_lectures = []

for idx, row in train_df.iterrows():
    # Skip if no tags
    if not row['tags']:
        watched_lectures.append(0)
        continue
    
    # Get lectures for this student with any matching tag
    student_lectures = combine_df[
        (combine_df['student_id'] == row['student_id']) &
        (combine_df['tags'].isin(row['tags'])) &  # Now comparing integers to integers
        (combine_df['timestamp'] < row['timestamp'])
    ]
    
    watched_lectures.append(1 if len(student_lectures) > 0 else 0)

train_df['watched_lecture'] = watched_lectures